# Lecture 05 Deep Learning with Keras

## Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [27]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
LECTURE_ID = "05"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", LECTURE_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Building an Image Classifier

First let's import TensorFlow and Keras.

In [28]:
import tensorflow as tf
from tensorflow import keras

In [29]:
tf.__version__

'2.1.0'

In [30]:
keras.__version__

'2.2.4-tf'

# Exercise 

*Exercise: Train a deep MLP on the MNIST dataset (you can load it using `keras.datasets.mnist.load_data()`. See if you can get over 98% precision. Try adding all the bells and whistles—save checkpoints, use early stopping, and plot learning curves using TensorBoard.*

Steps to carry out:
- load the dataset 
- explore dataset and normalise it
- define the NN model
- compile the model
- evaluate the model


## Load the dataset

In [31]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

In [32]:
X_train_full.shape
y_train_full.shape

(60000,)

## Validation set & Normalisation 

In [33]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

## Model

In [34]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
# keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")
model.add(keras.layers.Dense(300, activation="relu",kernel_initializer="he_normal"))
model.add(keras.layers.Dense(100, activation="relu",kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [35]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [36]:
# to plot

## Compile

In [37]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=["accuracy"])

In [41]:
run_index = 1
run_logdir = os.path.join(os.curdir, 'myminst_logs','run_{:03d}'.format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 5s 97us/sample - loss: 0.5851 - accuracy: 0.8450 - val_loss: 0.2950 - val_accuracy: 0.9194
Epoch 2/30
55000/55000 [==============================] - 6s 102us/sample - loss: 0.2764 - accuracy: 0.9215 - val_loss: 0.2385 - val_accuracy: 0.9320
Epoch 3/30
55000/55000 [==============================] - 5s 96us/sample - loss: 0.2258 - accuracy: 0.9358 - val_loss: 0.1960 - val_accuracy: 0.9432
Epoch 4/30
55000/55000 [==============================] - 6s 106us/sample - loss: 0.1929 - accuracy: 0.9453 - val_loss: 0.1734 - val_accuracy: 0.9528
Epoch 5/30
55000/55000 [==============================] - 5s 98us/sample - loss: 0.1682 - accuracy: 0.9522 - val_loss: 0.1565 - val_accuracy: 0.9574
Epoch 6/30
55000/55000 [==============================] - 5s 90us/sample - loss: 0.1496 - accuracy: 0.9574 - val_loss: 0.1428 - val_accuracy: 0.9624
Epoch 7/30
55000/55000 [==============================]

In [42]:
# %load_ext tensorboard
# %tensorboard --logdir=./myminst_logs --port=6006

In [43]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 57us/sample - loss: 0.0712 - accuracy: 0.9780


[0.07118143979699816, 0.978]